<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from openmdao.api import ImplicitComponent
import numpy as np
# from scipy.linalg import lu_factor, lu_solve
from scipy.sparse.linalg import gmres as gmres_solve

class LinearSysComp(ImplicitComponent):
    """
    A Simple Implicit Component representing a Linear System Kd=f
    
    """
    def initialize(self):
        self.metadata.declare('num_elements', types=int)
        
    def setup(self):
        num_elements = self.metadata['num_elements']
        size = num_elements
        self.add_input('K', shape = (size,size))
        self.add_input('f', shape = size)
        self.add_output('d', shape = size)
        rows = np.outer(np.arange(size), np.ones(size, int)).flatten()
        cols = np.arange(size ** 2)
        self.declare_partials('d', 'K', rows=rows, cols=cols)
        self.declare_partials('d', 'd')
        
    def apply_nonlinear(self, inputs, outputs, residuals):
        residuals['d'] = np.dot(inputs['K'], outputs['d']) - inputs['f']
    
    def linearize(self, inputs, outputs, partials):
        num_elements = self.metadata['num_elements']
        size = num_elements
        #self.lu = lu_factor(inputs['K'])
        partials['d', 'K'] = np.outer(np.ones(size), outputs['d']).flatten()
        partials['d', 'd'] = inputs['K']
        
    def solve_nonlinear(self, inputs, outputs):
        #self.lu = lu_factor(inputs['K'])
        # outputs['d'] = lu_solve(self.lu, inputs['f'])
        outputs['d'] = gmres_solve(inputs['K'],inputs['f'])[0]
        
    def solve_linear(self, d_outputs, d_residuals, mode):
        if mode == 'fwd':
            # d_outputs['d'] = lu_solve(self.lu, d_residuals['d'], trans=0)
            d_outputs['d'] = gmres_solve(inputs['K'], d_residuals['d'], trans=0)
        else:
            # d_residuals['d'] = lu_solve(self.lu, d_outputs['d'], trans=1)
            d_residuals['d'] = gmres_solve(inputs['K'], d_outputs['d'], trans=1)

In [2]:
from openmdao.api import Problem, Group, ImplicitComponent, IndepVarComp, NonlinearBlockGS, PETScKrylov
prob = Problem()
model = prob.model = Group()

np.random.seed(0)
num_elements = 400
model.add_subsystem('pK', IndepVarComp('K', np.random.rand(num_elements,num_elements)))
model.add_subsystem('pf', IndepVarComp('f', np.eye(num_elements,1)))
model.add_subsystem('Kdeqf', LinearSysComp(num_elements=num_elements))
model.connect('pK.K', 'Kdeqf.K')
model.connect('pf.f', 'Kdeqf.f')

model.nonlinear_solver = NonlinearBlockGS()

model.linear_solver = PETScKrylov()

prob.setup()
prob.run_model()
prob['Kdeqf.d']

NL: NLBGS Failed to Converge in 10 iterations


array([  9.76742203e-04,   2.70917275e-03,   3.24417794e-03,
         1.01071492e-03,  -1.39230862e-03,  -2.08210070e-04,
        -3.19847038e-03,  -9.81246667e-05,  -1.92833707e-03,
         1.82071782e-03,   9.17469727e-04,  -2.33510839e-04,
         1.60408949e-03,   4.19901742e-04,   7.63418658e-04,
        -5.59275261e-04,   2.34319211e-03,   3.78355282e-03,
         6.65660294e-05,  -9.36996499e-04,   1.22078200e-03,
        -1.32109237e-03,  -1.44065572e-03,   1.28161512e-03,
        -8.52707736e-05,  -1.24708886e-03,   5.27186919e-05,
         8.43639585e-04,   1.66815789e-04,  -1.18757910e-03,
        -1.19280008e-03,  -2.19392793e-04,  -1.21515042e-03,
        -2.99010058e-04,   1.88474468e-03,   9.30166980e-04,
         2.05193858e-03,   1.59423550e-03,  -6.04506760e-04,
        -2.09950644e-03,   1.07095876e-03,   1.32407680e-03,
         1.90535397e-03,  -2.88521999e-03,  -1.05868856e-03,
        -1.13827832e-03,   1.29176295e-03,  -7.96143895e-04,
         3.13970791e-03,

In [3]:
np.random.seed(0)
K = np.random.rand(400,400)
f = np.eye(400,1)
d = gmres_solve(K,f)[0]
d

array([  9.76742203e-04,   2.70917275e-03,   3.24417794e-03,
         1.01071492e-03,  -1.39230862e-03,  -2.08210070e-04,
        -3.19847038e-03,  -9.81246667e-05,  -1.92833707e-03,
         1.82071782e-03,   9.17469727e-04,  -2.33510839e-04,
         1.60408949e-03,   4.19901742e-04,   7.63418658e-04,
        -5.59275261e-04,   2.34319211e-03,   3.78355282e-03,
         6.65660294e-05,  -9.36996499e-04,   1.22078200e-03,
        -1.32109237e-03,  -1.44065572e-03,   1.28161512e-03,
        -8.52707736e-05,  -1.24708886e-03,   5.27186919e-05,
         8.43639585e-04,   1.66815789e-04,  -1.18757910e-03,
        -1.19280008e-03,  -2.19392793e-04,  -1.21515042e-03,
        -2.99010058e-04,   1.88474468e-03,   9.30166980e-04,
         2.05193858e-03,   1.59423550e-03,  -6.04506760e-04,
        -2.09950644e-03,   1.07095876e-03,   1.32407680e-03,
         1.90535397e-03,  -2.88521999e-03,  -1.05868856e-03,
        -1.13827832e-03,   1.29176295e-03,  -7.96143895e-04,
         3.13970791e-03,